## Data Preperation


In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import csv
import lxml.html as lh
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.datasets.samples_generator import make_blobs


# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Libraries imported.')




Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 1- Location

First let's download the location data and read it into a pandas dataframe.

>**Data source:** London data store - Greater London Authority  
**Source link:** https://data.london.gov.uk/dataset/postcode-directory-for-london  
**Data details:** Postcode, Latitude, Longtitude, LSOA*  
(* *A Lower Layer Super Output Area (LSOA) is a geographic area. Lower Layer Super Output Areas are a geographic hierarchy designed to improve the reporting of small area statistics in England and Wales.*)

In [27]:
location=pd.read_csv("https://data.london.gov.uk/download/postcode-directory-for-london/fd269535-973a-418f-8847-da405687e2e2/London_postcode-ONS-postcode-Directory-May15.csv")
location.head()

,pcd,pcd2,pcds,dointr,doterm,oscty,oslaua,osward,usertype,oseast1m,osnrth1m,osgrdind,oshlthau,hro,ctry,gor,streg,pcon,eer,teclec,ttwa,pct,nuts,psed,cened,edind,oshaprev,lea,oldha,wardc91,wardo91,ward98,statsward,oa01,casward,park,lsoa01,msoa01,ur01ind,oac01,oldpct,oa11,lsoa11,msoa11,parish,wz11,ccg,bua11,buasd11,ru11ind,oac11,lat,long
0,BR1 1AA,BR1 1AA,BR1 1AA,199002,200812,E99999999,E09000006,E05000109,1,540260,169398,1,E18000007,E19000003,E92000001,E12000007,5,E14000604,E15000007,E24000016,E30000082,E16000004,E05000109,01AFFR19,AUFR19,1,Q07,30,QEY,02AUFR,01AFFR,00AFFR,00AFGG,E00003255,00AFG,E99999999,E01000676,E02000134,5,7A3,5A7,E00003255,E01000676,E02000134,E43000196,E33029192,E38000023,E34004707,E35000434,,2D1,51.406271,0.015177
1,BR1 1AB,BR1 1AB,BR1 1AB,201203,,E99999999,E09000006,E05000109,0,540260,169398,1,E18000007,E19000003,E92000001,E12000007,5,E14000604,E15000007,E24000016,E30000082,E16000004,E05000109,01AFFR19,AUFR19,1,Q07,30,QEY,02AUFR,01AFFR,00AFFR,00AFGG,E00003255,00AFG,E99999999,E01000676,E02000134,5,7A3,5A7,E00003255,E01000676,E02000134,E43000196,E33029192,E38000023,E34004707,E35000434,,2D1,51.406271,0.015177
2,BR1 1AD,BR1 1AD,BR1 1AD,201409,,E99999999,E09000006,E05000109,1,540383,168708,1,E18000007,E19000003,E92000001,E12000007,5,E14000604,E15000007,E24000016,E30000082,E16000004,E05000109,01AFGA17,AUGA17,1,Q07,30,QEY,02AUGA,01AFGA,00AFGA,00AFGG,E00003264,00AFG,E99999999,E01000675,E02000144,5,2B2,5A7,E00003264,E01000675,E02000144,E43000196,E33029199,E38000023,E34004707,E35000434,,4C3,51.400040,0.016671
3,BR1 1AE,BR1 1AE,BR1 1AE,200808,,E99999999,E09000006,E05000109,0,540194,169201,1,E18000007,E19000003,E92000001,E12000007,5,E14000604,E15000007,E24000016,E30000082,E16000004,E05000109,01AFFR19,AUFR19,1,Q07,30,QEY,02AUFR,01AFFR,00AFFR,00AFGG,E00003266,00AFG,E99999999,E01000677,E02000144,5,2B2,5A7,E00003266,E01000677,E02000144,E43000196,E33029200,E38000023,E34004707,E35000434,,2D1,51.404517,0.014151
4,BR1 1AX,BR1 1AX,BR1 1AX,200106,200610,E99999999,E09000006,E05000109,1,540421,169620,1,E18000007,E19000003,E92000001,E12000007,5,E14000604,E15000007,E24000016,E30000082,E16000004,E05000109,01AFFX24,AUFX24,1,Q07,30,QEY,02AUFX,01AFFX,00AFFX,00AFGG,E00003250,00AFG,E99999999,E01000676,E02000134,5,2A2,5A7,E00003250,E01000676,E02000134,E43000196,E33029181,E38000023,E34004707,E35000434,,2D1,51.408226,0.017578


Let's drop unnecessary columns

In [28]:
location.rename(columns = {'pcd': 'Postcode','oa11': 'LSOA2','lsoa11': 'LSOA', 'lat' : 'Latitude', 'long': 'Longitude'} ,inplace = True)
location=location[['Postcode','LSOA','LSOA2', 'Latitude', 'Longitude']]
location.head()


,Postcode,LSOA,LSOA2,Latitude,Longitude
0,BR1 1AA,E01000676,E00003255,51.406271,0.015177
1,BR1 1AB,E01000676,E00003255,51.406271,0.015177
2,BR1 1AD,E01000675,E00003264,51.400040,0.016671
3,BR1 1AE,E01000677,E00003266,51.404517,0.014151
4,BR1 1AX,E01000676,E00003250,51.408226,0.017578


In [29]:
location.describe()

,Latitude,Longitude
count,312430.000000,312430.000000
mean,51.507481,-0.132311
std,0.069363,0.140011
min,51.293185,-0.499219
25%,51.466258,-0.208942
50%,51.513973,-0.132455
75%,51.552555,-0.059694
max,51.688813,0.321160


In [32]:
location = location.sort_values('LSOA', ascending=False)
location= location.drop_duplicates(subset='LSOA', keep='first')
location.describe()

,Latitude,Longitude
count,4835.000000,4835.000000
mean,51.505940,-0.117859
std,0.075536,0.155006
min,51.298747,-0.496741
25%,51.452714,-0.217414
50%,51.513052,-0.113939
75%,51.562537,-0.015759
max,51.682138,0.309367


In [6]:
location.to_csv('location.csv')
print("csv saved")

csv saved


### 2- Crime

>**Data source:** London data store , Greater London Authority   
**Source link:** https://data.london.gov.uk/dataset/recorded_crime_summary  
**Data details:**  MPS Borough Level Crime (most recent 24 months) , MPS LSOA Level Crime (most recent 24 months) 

In [7]:
crime=pd.read_csv("https://data.london.gov.uk/download/recorded_crime_summary/644a9e0d-75a3-4c3a-91ad-03d2a7cb8f8e/MPS%20LSOA%20Level%20Crime%20%28most%20recent%2024%20months%29.csv")
crime.head()

,LSOA Code,Borough,Major Category,Minor Category,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909
0,E01000006,Barking and Dagenham,Arson and Criminal Damage,Arson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,E01000007,Barking and Dagenham,Arson and Criminal Damage,Arson,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,E01000008,Barking and Dagenham,Arson and Criminal Damage,Arson,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,E01000009,Barking and Dagenham,Arson and Criminal Damage,Arson,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,E01000010,Barking and Dagenham,Arson and Criminal Damage,Arson,0,0,0,0,0,0,0,0,2,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0


In [8]:
crime.rename(columns = {'LSOA Code': 'LSOA'} ,inplace = True)
crime['Sum_Crime'] =crime.sum(axis=1)
crime=crime [['Borough','LSOA','Sum_Crime']]
crime.head()

,Borough,LSOA,Sum_Crime
0,Barking and Dagenham,E01000006,1
1,Barking and Dagenham,E01000007,1
2,Barking and Dagenham,E01000008,2
3,Barking and Dagenham,E01000009,2
4,Barking and Dagenham,E01000010,6


In [9]:
crime_2=crime.groupby(['LSOA'])['Sum_Crime'].sum().reset_index()
crime_2.head()

,LSOA,Sum_Crime
0,E01000006,203
1,E01000007,868
2,E01000008,399
3,E01000009,500
4,E01000010,2669


In [10]:
crime_2.describe()

,Sum_Crime
count,4829.000000
mean,343.855664
std,532.046147
min,43.000000
25%,167.000000
50%,237.000000
75%,360.000000
max,14935.000000


In order to calculate crime per 1000 person, we will pull LSOA level population data

>**Data source:** London data store , Greater London Authority  
**Source link:** https://data.london.gov.uk/dataset/lsoa-atlas  
**Data details:**  LSOA Level population data to calculate crime rate per population ,Current LSOA boundaries post 2011

In [11]:
population=pd.read_excel("https://data.london.gov.uk/download/lsoa-atlas/b8e01c3a-f5e3-4417-82b3-02ad271e6ee8/lsoa-data.xls", header=1)
population.head()

,Unnamed: 0,Unnamed: 1,All Ages,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Aged 0-15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Aged 16-29,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Aged 30-44,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Aged 45-64,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Aged 65+,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Working-age,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Age Structure,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Area (Hectares),Persons per hectare,Unnamed: 102,All households,Couple household with dependent children,Couple household without dependent children,Lone parent household,One person household,Other household Types,% Couple household with dependent children,% Couple household without dependent children,% Lone parent household,% One person household,% Other multi person household,White,Mixed/multiple ethnic groups,Asian/Asian British,Black/African/Caribbean/Black British,Other ethnic group,BAME,White (%),Mixed/multiple ethnic groups (%),Asian/Asian British (%),Black/African/Caribbean/Black British (%),Other ethnic group (%),BAME (%),United Kingdom,Not United Kingdom,% United Kingdom,% Not United Kingdom,Households with at least one person aged 16 or over with English as a main language,Households where no people aged 16 or over have English as a main language,% of households with at least one person aged 16 or over with English as a main language,% of households where no people aged 16 or over have English as a main language,Christian,Buddhist,Hindu,Jewish,Muslim,Sikh,Other religion,No religion,Religion not stated,Christian (%),Buddhist (%),Hindu (%),Jewish (%),Muslim (%),Sikh (%),Other religion (%),No religion (%),Religion not stated (%),Owned outright,Owned with a mortgage or loan,Social rented,Private rented,Owned outright (%),Owned with a mortgage or loan (%),Social rented (%),Private rented (%),All Households,Household spaces with at least one usual resident,Household spaces with no usual residents,Whole house or bungalow: Detached,Whole house or bungalow: Semi-detached,Whole house or bungalow: Terraced (including end-terrace),"Flat, maisonette or apartment",Household spaces with at least one usual resident (%),Household spaces with no usual residents (%),Whole house or bungalow: Detached (%),Whole house or bungalow: Semi-detached (%),Whole house or bungalow: Terraced (including end-terrace) (%),"Flat, maisonette or apartment (%)",Median Price (£),Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Sales,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184
0,Codes,Names,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,All Ages,0-15,16-29,30-44,45-64,65+,Working-age,NaN,2012.000000,2013.000000,2011,2011,2011,2011,2011,2011,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011,2011,2011,2011,2011,2011,2011.000000,2011.000000,2011.00

In [12]:
population=population[['Unnamed: 0', 'Unnamed: 14']]
population=population.drop(population.index[0]).reset_index(drop=True)
population.rename(columns = {'Unnamed: 0' : 'LSOA','Unnamed: 14': 'Population'} ,inplace = True)
population.head()

,LSOA,Population
0,E01000907,1466
1,E01000908,1647
2,E01000909,1642
3,E01000912,1653
4,E01000913,1827


In [13]:
population.describe()

,Population
count,4835.000000
mean,1740.751810
std,304.551285
min,965.000000
25%,1550.000000
50%,1690.000000
75%,1878.000000
max,6139.000000


Lets calculate crime per person for each LSOA

In [14]:
crime_per_pop= pd.merge(crime_2, population, how='left', on='LSOA')
crime_per_pop.head(10)

,LSOA,Sum_Crime,Population
0,E01000006,203,1714
1,E01000007,868,1525
2,E01000008,399,1595
3,E01000009,500,1737
4,E01000010,2669,3199
5,E01000011,192,1596
6,E01000012,262,1937
7,E01000013,632,1684
8,E01000014,234,1645
9,E01000015,238,2404


In [15]:
crime_per_pop.describe()
crime_per_pop.to_csv('crime.csv')
print("saved")

saved


In [16]:
crime_per_pop ['crime_per_1000'] = crime_per_pop ['Sum_Crime'] / crime_per_pop ['Population'] *1000 /2
crime_per_pop=crime_per_pop.drop(columns=['Sum_Crime','Population'])
crime_per_pop.head()

,LSOA,crime_per_1000
0,E01000006,59.218203
1,E01000007,284.590164
2,E01000008,125.078370
3,E01000009,143.926310
4,E01000010,417.161613


merge with Post code data

In [17]:
crime_per_pop= pd.merge(crime_per_pop, location, how='left', on='LSOA')
crime_per_pop.head(10)

,LSOA,crime_per_1000,Postcode,LSOA2,Latitude,Longitude
0,E01000006,59.218203,IG119TH,E00000049,51.540223,0.086372
1,E01000007,284.590164,IG118JD,E00000040,51.538691,0.075934
2,E01000008,125.078370,IG118LT,E00000048,51.541259,0.071061
3,E01000009,143.926310,IG118EA,E00000054,51.539282,0.081628
4,E01000010,417.161613,IG118ZB,E00000062,51.537377,0.079018
5,E01000011,60.150376,IG118RH,E00000064,51.543769,0.078603
6,E01000012,67.630356,IG118ND,E00000045,51.542122,0.072615
7,E01000013,187.648456,RM9 5RL,E00000096,51.542382,0.146624
8,E01000014,71.124620,RM9 5DT,E00000085,51.545875,0.143024
9,E01000015,49.500832,RM108ED,E00000080,51.544118,0.157898


In [18]:
crime_per_pop.to_csv('crime_per_pop.csv')
crime_per_pop.describe()

,crime_per_1000,Latitude,Longitude
count,4829.000000,4829.000000,4829.000000
mean,96.590116,51.505927,-0.117892
std,153.278908,0.075582,0.155099
min,15.472779,51.298747,-0.496741
25%,50.326797,51.452695,-0.217431
50%,69.258809,51.512905,-0.114275
75%,100.428495,51.562570,-0.015706
max,6259.430008,51.682138,0.309367


In [19]:
#obtain latitude and longitude for London
address = 'London'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of London are 51.5073219, -0.1276474.


In [20]:
london_crime_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, Postcode, Crime in zip(crime_per_pop['Latitude'], crime_per_pop['Longitude'], crime_per_pop['Postcode'], crime_per_pop['crime_per_1000']):
    label = '{}: Crime: {}'.format(Postcode, Crime)
    label = folium.Popup(label, parse_html=True)
    if int(Crime) < 69:
        color="green" 
    elif int(Crime) >  100:
        color = "red"
    else: 
        color="yellow"
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color=color,
        fill=True,
        fill_opacity=0.7,
        parse_html=False).add_to(london_crime_map)  
    

london_crime_map
london_crime_map.save('index.html')

### 3-Commuting duration to Bank station

>**Data source:** London data store , Greater London Authority   
**Source link:** https://data.london.gov.uk/download/mylondon/c2e9ebc1-935b-460c-9361-293398d84fe5/MyLondon_traveltime_to_Bank_station_OA.csv  
**Data details:** 

In [34]:
commute=pd.read_csv("https://data.london.gov.uk/download/mylondon/c2e9ebc1-935b-460c-9361-293398d84fe5/MyLondon_traveltime_to_Bank_station_OA.csv")
commute.head()

,OA11CD,driving_distance_miles,driving_time_mins,public_transport_time_mins,cycling_distance_miles,cycling_time_mins,walking_distance_miles,walking_time_mins
0,E00000001,0.86,6,16,0.83,6,0.80,16
1,E00000003,0.99,7,14,0.98,7,0.73,14
2,E00000005,0.86,6,13,0.83,6,0.65,13
3,E00000007,0.64,5,10,0.61,4,0.51,10
4,E00000010,0.93,6,17,0.88,6,0.85,17


In [35]:
commute.rename(columns = {'OA11CD': 'LSOA2'} ,inplace = True)
commute=commute[['LSOA2', 'public_transport_time_mins']]
commute.head()

,LSOA2,public_transport_time_mins
0,E00000001,16
1,E00000003,14
2,E00000005,13
3,E00000007,10
4,E00000010,17


In [36]:
commute.to_csv('commute.csv')
commute.describe()

,public_transport_time_mins
count,25053.000000
mean,51.111763
std,15.215065
min,0.000000
25%,40.000000
50%,50.000000
75%,61.000000
max,120.000000


In [44]:
df3= pd.merge(crime_per_pop, commute, how='left', on='LSOA2') 
df3.head(10)

,LSOA,crime_per_1000,Postcode,LSOA2,Latitude,Longitude,public_transport_time_mins
0,E01000006,59.218203,IG119TH,E00000049,51.540223,0.086372,44
1,E01000007,284.590164,IG118JD,E00000040,51.538691,0.075934,44
2,E01000008,125.078370,IG118LT,E00000048,51.541259,0.071061,48
3,E01000009,143.926310,IG118EA,E00000054,51.539282,0.081628,40
4,E01000010,417.161613,IG118ZB,E00000062,51.537377,0.079018,41
5,E01000011,60.150376,IG118RH,E00000064,51.543769,0.078603,46
6,E01000012,67.630356,IG118ND,E00000045,51.542122,0.072615,48
7,E01000013,187.648456,RM9 5RL,E00000096,51.542382,0.146624,51
8,E01000014,71.124620,RM9 5DT,E00000085,51.545875,0.143024,55
9,E01000015,49.500832,RM108ED,E00000080,51.544118,0.157898,55


In [45]:
df3.describe()

,crime_per_1000,Latitude,Longitude,public_transport_time_mins
count,4829.000000,4829.000000,4829.000000,4829.000000
mean,96.590116,51.505927,-0.117892,51.622696
std,153.278908,0.075582,0.155099,15.156123
min,15.472779,51.298747,-0.496741,0.000000
25%,50.326797,51.452695,-0.217431,40.000000
50%,69.258809,51.512905,-0.114275,51.000000
75%,100.428495,51.562570,-0.015706,61.000000
max,6259.430008,51.682138,0.309367,119.000000


### 4-Nearby Parks and supermarkets


#### Parks
>**Data source:** Foursquare API  
**Foursquare Category Id:** 4bf58dd8d48988d163941735  


#### Supermarkets
>**Data source:** Foursquare API  
**Foursquare Category Id:** Supermarket :52f2ab2ebcbc57f1066b8b46 , Grocery store: 4bf58dd8d48988d118951735, Market: 50be8ee891d4fa8dcc7199a7  

In [46]:
df3['Park'] = ''
df3['Supermarket'] = ''
df3.head()

,LSOA,crime_per_1000,Postcode,LSOA2,Latitude,Longitude,public_transport_time_mins,Park,Supermarket
0,E01000006,59.218203,IG119TH,E00000049,51.540223,0.086372,44,,
1,E01000007,284.590164,IG118JD,E00000040,51.538691,0.075934,44,,
2,E01000008,125.078370,IG118LT,E00000048,51.541259,0.071061,48,,
3,E01000009,143.926310,IG118EA,E00000054,51.539282,0.081628,40,,
4,E01000010,417.161613,IG118ZB,E00000062,51.537377,0.079018,41,,


In [69]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

neighborhood_latitude = df3['Latitude'][0] # neighborhood latitude value
neighborhood_longitude = df3['Longitude'][0] # neighborhood longitude value

#neighborhood_name = df2['Neighbourhood\n'][0] # neighborhood name 
    # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QPYKIY1P2ITZCBSX4CIY5ZHUVVJ4J1WUJSA21331P4GBB2O5&client_secret=K0AS5USKU3VZIWCVTON3NWP5NYXDU4VJB3RVCZK3YGBPCTTZ&v=20180604&ll=51.540223,0.086372&radius=500&limit=100'